In [ ]:
!pip install catboost

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.linear_model import Lars, LassoLars, OrthogonalMatchingPursuit
from sklearn.linear_model import BayesianRidge, ARDRegression, PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor, TheilSenRegressor, HuberRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
#from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import HuberRegressor, LinearRegression


In [2]:
train = pd.read_csv('final_train.csv')
test = pd.read_csv('final_test.csv')
building = pd.read_csv('../building_info.csv')
submit = pd.read_csv('../sample_submission.csv')

In [ ]:
building = building.replace('-', np.NaN)
building[['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']] = building[['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']].astype(float)

building['태양광용량(kW)'] = building['태양광용량(kW)'].fillna(0)
building['태양광용량(kW)'] = np.where(building['태양광용량(kW)'] > 0, 1, 0)

train = pd.merge(train, building[['건물번호', '건물유형', '태양광용량(kW)']], how = 'outer',on = '건물번호')
test = pd.merge(test, building[['건물번호', '건물유형', '태양광용량(kW)']], how = 'outer',on = '건물번호')

date = pd.to_datetime(train.일시)
train['hour'] = date.dt.hour
train['day'] = date.dt.weekday
train['month'] = date.dt.month
train['week'] = date.dt.weekofyear

date = pd.to_datetime(test.일시)
test['hour'] = date.dt.hour
test['day'] = date.dt.weekday
test['month'] = date.dt.month
test['week'] = date.dt.weekofyear

for month in train['month'].unique():
    train.loc[train['month'] == month, ['강수량(mm)', '풍속(m/s)', '습도(%)']] = train.loc[train['month'] == month, ['강수량(mm)', '풍속(m/s)', '습도(%)']].fillna(train.loc[train['month'] == month, ['강수량(mm)', '풍속(m/s)', '습도(%)']].mean())

train = train.fillna(0)
test = test.fillna(0)

train.loc[(train['건물번호'] == 95) & (train['전력소비량(kWh)'] < 1), '전력소비량(kWh)'] = np.NaN
train['전력소비량(kWh)'] = train['전력소비량(kWh)'].interpolate(limit_direction='both', method='linear').round(3)

power_mean = pd.pivot_table(train, values = '전력소비량(kWh)', index = ['건물번호', 'day', 'month'], aggfunc = np.mean).reset_index()
train['month_day_mean'] = train.apply(lambda x : power_mean.loc[(power_mean.건물번호 == x['건물번호']) & (power_mean.day == x['day']) & (power_mean.month == x['month']) ,'전력소비량(kWh)'].values[0], axis = 1)
test['month_day_mean'] = test.apply(lambda x : power_mean.loc[(power_mean.건물번호 == x['건물번호']) & (power_mean.day == x['day']) & (power_mean.month == x['month']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_mean = pd.pivot_table(train, values = '전력소비량(kWh)', index = ['건물번호', 'hour', 'day'], aggfunc = np.mean).reset_index()
train['day_hour_mean'] = train.apply(lambda x : power_mean.loc[(power_mean.건물번호 == x['건물번호']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)
test['day_hour_mean'] = test.apply(lambda x : power_mean.loc[(power_mean.건물번호 == x['건물번호']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_std = pd.pivot_table(train, values = '전력소비량(kWh)', index = ['건물번호', 'hour', 'day'], aggfunc = np.std).reset_index()
train['day_hour_std'] = train.apply(lambda x : power_std.loc[(power_std.건물번호 == x['건물번호']) & (power_std.hour == x['hour']) & (power_std.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)
test['day_hour_std'] = test.apply(lambda x : power_std.loc[(power_std.건물번호 == x['건물번호']) & (power_std.hour == x['hour']) & (power_std.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_hour_mean = pd.pivot_table(train, values = '전력소비량(kWh)', index = ['건물번호', 'hour'], aggfunc = np.mean).reset_index()
train['hour_mean'] = train.apply(lambda x : power_hour_mean.loc[(power_hour_mean.건물번호 == x['건물번호']) & (power_hour_mean.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)
test['hour_mean'] = test.apply(lambda x : power_hour_mean.loc[(power_hour_mean.건물번호 == x['건물번호']) & (power_hour_mean.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_hour_std = pd.pivot_table(train, values = '전력소비량(kWh)', index = ['건물번호', 'hour'], aggfunc = np.std).reset_index()
train['hour_std'] = train.apply(lambda x : power_hour_std.loc[(power_hour_std.건물번호 == x['건물번호']) & (power_hour_std.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)
test['hour_std'] = test.apply(lambda x : power_hour_std.loc[(power_hour_std.건물번호 == x['건물번호']) & (power_hour_std.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)


power_mean = pd.pivot_table(train, values = '전력소비량(kWh)', index = ['건물유형', 'hour', 'day'], aggfunc = np.mean).reset_index()
train['type_day_hour_mean'] = train.apply(lambda x : power_mean.loc[(power_mean.건물유형 == x['건물유형']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)
test['type_day_hour_mean'] = test.apply(lambda x : power_mean.loc[(power_mean.건물유형 == x['건물유형']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_std = pd.pivot_table(train, values = '전력소비량(kWh)', index = ['건물유형', 'hour', 'day'], aggfunc = np.std).reset_index()
train['type_day_hour_std'] = train.apply(lambda x : power_std.loc[(power_std.건물유형 == x['건물유형']) & (power_std.hour == x['hour']) & (power_std.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)
test['type_day_hour_std'] = test.apply(lambda x : power_std.loc[(power_std.건물유형 == x['건물유형']) & (power_std.hour == x['hour']) & (power_std.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_hour_mean = pd.pivot_table(train, values = '전력소비량(kWh)', index = ['건물유형', 'hour'], aggfunc = np.mean).reset_index()
train['type_hour_mean'] = train.apply(lambda x : power_hour_mean.loc[(power_hour_mean.건물유형 == x['건물유형']) & (power_hour_mean.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)
test['type_hour_mean'] = test.apply(lambda x : power_hour_mean.loc[(power_hour_mean.건물유형 == x['건물유형']) & (power_hour_mean.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_hour_std = pd.pivot_table(train, values = '전력소비량(kWh)', index = ['건물유형', 'hour'], aggfunc = np.std).reset_index()
train['type_hour_std'] = train.apply(lambda x : power_hour_std.loc[(power_hour_std.건물유형 == x['건물유형']) & (power_hour_std.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)
test['type_hour_std'] = test.apply(lambda x : power_hour_std.loc[(power_hour_std.건물유형 == x['건물유형']) & (power_hour_std.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)


### 공휴일 변수 추가
train['holiday'] = train.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
train.loc[('20220601' <= train.일시)&(train.일시 < '20220602'), 'holiday'] = 1
train.loc[('20220606' <= train.일시)&(train.일시 < '20220607'), 'holiday'] = 1
train.loc[('20220815' <= train.일시)&(train.일시 < '20220816'), 'holiday'] = 1

test['holiday'] = test.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)

train['sin_time'] = np.sin(2*np.pi*train.hour/24)
train['cos_time'] = np.cos(2*np.pi*train.hour/24)
test['sin_time'] = np.sin(2*np.pi*test.hour/24)
test['cos_time'] = np.cos(2*np.pi*test.hour/24)

train['THI'] = 9/5*train['기온(C)'] - 0.55*(1-train['습도(%)']/100)*(9/5*train['습도(%)']-26)+32
test['THI'] = 9/5*test['기온(C)'] - 0.55*(1-test['습도(%)']/100)*(9/5*test['습도(%)']-26)+32

def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

train['CDH'] = 0
for num in range(1,101,1):
    temp = train[train['건물번호'] == num]
    cdh = CDH(temp['기온(C)'].values)
    train.loc[train['건물번호'] == num, 'CDH'] = cdh

test['CDH'] = 0
for num in range(1,101,1):
    temp = test[test['건물번호'] == num]
    cdh = CDH(temp['기온(C)'].values)
    test.loc[test['건물번호'] == num, 'CDH'] = cdh

train['7_shifted_전력소비량'] = train['전력소비량(kWh)'].shift(24*7)
train = train[train.일시 >= '20220608'].reset_index(drop=True)

for i in train['건물번호'].unique():
    test.loc[test['건물번호'] == i, '7_shifted_전력소비량'] = train.loc[train['건물번호'] == i, '전력소비량(kWh)'][-7*24:].values

train_x = train.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)', '건물유형'])
train_x_type = train.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
train_y = train['전력소비량(kWh)']

test_x = test[train_x.columns]
test_x_type = test[train_x_type.columns]
train_x.head()

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),태양광용량(kW),hour,day,month,week,...,type_day_hour_mean,type_day_hour_std,type_hour_mean,type_hour_std,holiday,sin_time,cos_time,THI,CDH,7_shifted_전력소비량
0,1,19.0,1.889045,3.2,61.0,0,0,2,6,23,...,1620.812462,886.546805,1616.129012,909.159339,0,0.000000,1.000000,48.2249,-38.9,1085.28
1,1,18.8,1.889045,2.6,61.0,0,1,2,6,23,...,1608.544000,906.029700,1603.843635,921.827876,0,0.258819,0.965926,47.8649,-45.1,1047.36
2,1,18.5,1.889045,2.6,62.0,0,2,2,6,23,...,1577.389538,900.729730,1575.108376,916.043890,0,0.500000,0.866025,47.4096,-51.4,974.88
3,1,18.1,1.889045,2.2,63.0,0,3,2,6,23,...,1561.107077,900.693834,1553.335094,908.635022,0,0.707107,0.707107,46.7941,-58.0,953.76
4,1,17.2,1.889045,3.2,66.0,0,4,2,6,23,...,1546.894154,885.287208,1543.278965,900.520543,0,0.866025,0.500000,45.6064,-64.9,986.40


In [ ]:
'''
# 1
train.loc[770:777, '전력소비량(kWh)'] = train.loc[602:609, '전력소비량(kWh)'].values
train.loc[939:946, '전력소비량(kWh)'] = train.loc[602:609, '전력소비량(kWh)'].values

# 4
train.loc[7438:7439, '전력소비량(kWh)'] = train.loc[7416:7417, '전력소비량(kWh)'].values
train.loc[7440, '전력소비량(kWh)'] = np.nan

# 4
# train_df.loc[7438, '전력소비량(kWh)'] = train_df.loc[7438-24*7, '전력소비량(kWh)'].values

# 11
train.loc[21893, '전력소비량(kWh)'] = train.loc[21893-24*7, '전력소비량(kWh)']
train.loc[22058, '전력소비량(kWh)'] = train.loc[22058-24*7, '전력소비량(kWh)']
train.loc[22065, '전력소비량(kWh)'] = train.loc[22065-24*7, '전력소비량(kWh)']
train.loc[21700:21701, '전력소비량(kWh)'] = train.loc[21700-24*7:21701-24*7, '전력소비량(kWh)'].values
train.loc[22243, '전력소비량(kWh)'] = train.loc[22243-24*7, '전력소비량(kWh)']

# 17
train.loc[33888:33901, '전력소비량(kWh)'] = train.loc[33888-24*7:33901-24*7, '전력소비량(kWh)'].values

# 26
train.loc[52089:52090, '전력소비량(kWh)'] = train.loc[52089-24*7:52090-24*7, '전력소비량(kWh)'].values

# 28
train.loc[56450, '전력소비량(kWh)'] = train.loc[56450-24*7, '전력소비량(kWh)']

# 31
train.loc[61955, '전력소비량(kWh)'] = train.loc[61955-24*7, '전력소비량(kWh)']

# 34
train.loc[68973, '전력소비량(kWh)'] = train.loc[68973-24*7, '전력소비량(kWh)']

# 56
train.loc[112384, '전력소비량(kWh)'] = train.loc[112384-24*7, '전력소비량(kWh)']

# 61
train.loc[123132, '전력소비량(kWh)'] = train.loc[123132-24*7, '전력소비량(kWh)']

# 69
train.loc[138904, '전력소비량(kWh)'] = train.loc[138904-24*7, '전력소비량(kWh)']

# 72
train.loc[145424:145425, '전력소비량(kWh)'] = np.nan
train.loc[145786, '전력소비량(kWh)'] = np.nan
train.loc[145954, '전력소비량(kWh)'] = np.nan

# 95
train.loc[193120:193121, '전력소비량(kWh)'] = train.loc[193120-24*7:193121-24*7, '전력소비량(kWh)'].values
train.loc[193133, '전력소비량(kWh)'] = train.loc[193133-24*7, '전력소비량(kWh)']

# 75
train.loc[151303:151314, '전력소비량(kWh)'] = train.loc[151303-24*7:151314-24*7, '전력소비량(kWh)'].values
train.loc[151423:151455, '전력소비량(kWh)'] = train.loc[151423-24*7:151455-24*7, '전력소비량(kWh)'].values
train.loc[152393, '전력소비량(kWh)'] = train.loc[152393-24*7, '전력소비량(kWh)']

# 70
# train.loc[142200:142229, '전력소비량(kWh)'] = np.nan
train.loc[142004+169:142004+169+168, '전력소비량(kWh)'] = train.loc[142004:142004+168, '전력소비량(kWh)'].values
train.loc[142171:142172, '전력소비량(kWh)'] = train.loc[142171-24*7:142172-24*7, '전력소비량(kWh)'].values
train.loc[142340:142379, '전력소비량(kWh)'] = train.loc[142340-24*7:142379-24*7, '전력소비량(kWh)'].values
# train.loc[142200:142229, '전력소비량(kWh)'] = train.loc[142200-24*7:142229-24*7, '전력소비량(kWh)'].values

train['전력소비량(kWh)'] = train['전력소비량(kWh)'].interpolate(limit_direction='both', method='linear').round(3)
'''

In [ ]:
'''
info_series = building.set_index('건물번호')['연면적(m2)']
train['연면적'] = train['건물번호'].map(info_series)
test['연면적'] = test['건물번호'].map(info_series)
info_series = building.set_index('건물번호')['냉방면적(m2)']
train['냉방면적'] = train['건물번호'].map(info_series)
test['냉방면적'] = test['건물번호'].map(info_series)
info_series = building.set_index('건물번호')['태양광용량(kW)']
train['태양광용량'] = train['건물번호'].map(info_series)
test['태양광용량'] = test['건물번호'].map(info_series)
info_series = building.set_index('건물번호')['ESS저장용량(kWh)']
train['ESS저장용량'] = train['건물번호'].map(info_series)
test['ESS저장용량'] = test['건물번호'].map(info_series)
info_series = building.set_index('건물번호')['PCS용량(kW)']
train['PCS용량'] = train['건물번호'].map(info_series)
test['PCS용량'] = test['건물번호'].map(info_series)
train = train.replace('-', np.nan)
test = test.replace('-', np.nan)
train['태양광용량'] = train['태양광용량'].fillna(0)
train['ESS저장용량'] = train['ESS저장용량'].fillna(0)
train['PCS용량'] = train['PCS용량'].fillna(0)
test['태양광용량'] = test['태양광용량'].fillna(0)
test['ESS저장용량'] = test['ESS저장용량'].fillna(0)
test['PCS용량'] = test['PCS용량'].fillna(0)
train['태양광용량'] = train['태양광용량'].astype(float)
train['ESS저장용량'] = train['ESS저장용량'].astype(float)
train['PCS용량'] = train['PCS용량'].astype(float)
test['태양광용량'] = test['태양광용량'].astype(float)
test['ESS저장용량'] = test['ESS저장용량'].astype(float)
test['PCS용량'] = test['PCS용량'].astype(float)

## 날짜
train['일시'] = train['일시'].astype(str)
date = pd.to_datetime(train.일시, format='%Y%m%d %H')
train['hour'] = date.dt.hour
train['day'] = date.dt.weekday
train['month'] = date.dt.month
train['week'] = date.dt.isocalendar().week
test['일시'] = test['일시'].astype(str)
date = pd.to_datetime(test.일시, format='%Y%m%d %H')
test['hour'] = date.dt.hour
test['day'] = date.dt.weekday
test['month'] = date.dt.month
test['week'] = date.dt.isocalendar().week
train['week'] = train['week'].astype('int')
test['week'] = test['week'].astype('int')
for month in train['month'].unique():
    train.loc[train['month'] == month, ['풍속(m/s)', '습도(%)']] = train.loc[train['month'] == month, ['풍속(m/s)', '습도(%)']].fillna(train.loc[train['month'] == month, ['풍속(m/s)', '습도(%)']].mean())

train = train.fillna(0)
test = test.fillna(0)


## 위치
# 지역 이름을 매칭하는 규칙
location_rules = {
    ('20220801 13', 31.1, 2.4): "A",
    ('20220801 13', 31.1, 3.0): "B",
    ('20220801 13', 31.5, 2.4): "C",
    ('20220801 13', 31.5, 4.9): "D",
    ('20220801 13', 31.7, 1.9): "E",
    ('20220801 13', 31.7, 3.7): "F",
    ('20220801 13', 29.6, 3.9): "G",
    ('20220801 13', 29.6, 3.0): "H",
    ('20220801 13', 30.4, 3.7): "I",
    ('20220801 13', 30.4, 2.4): "J",
    ('20220801 13', 30.3): "K",
    ('20220801 13', 30.9): "L",
    ('20220801 13', 31.4, 2.9): "M",
    ('20220801 13', 31.4, 3.1): "N",
    ('20220801 13', 30.2): "O",
    ('20220801 13', 32.3): "P",
    ('20220801 13', 28.4): "Q",
    ('20220801 13', 31.8): "R",
    ('20220801 13', 29.4): "S",
    ('20220801 13', 29.1): "T",
    ('20220801 13', 29.0): "U",
    ('20220801 13', 32.6): "V",
    ('20220801 13', 29.7): "W",
    ('20220801 13', 26.2): "X",
    ('20220801 13', 30.5): "Y",
    ('20220801 13', 25.5): "Z",
    ('20220801 13', 26.5): "AA",
    ('20220801 13', 28.3): "BB"
}
train['location'] = ""
# 각 행에 대해 규칙을 적용
for idx, row in train.iterrows():
    date_time, temp, wind = row['일시'], row['기온(C)'], row['풍속(m/s)']
    # wind 값이 있는 경우
    if (date_time, temp, wind) in location_rules:
        train.loc[idx, 'location'] = location_rules[(date_time, temp, wind)]
    # wind 값이 없는 경우
    elif (date_time, temp) in location_rules:
        train.loc[idx, 'location'] = location_rules[(date_time, temp)]
# num에 따른 location mapping을 생성
num_location_mapping = train.loc[train['일시'] == '20220801 13', ['건물번호', 'location']].set_index('건물번호').to_dict()['location']
# 모든 행에 대해 num에 따른 location을 적용
train['location'] = train['건물번호'].map(num_location_mapping)
test['location'] = test['건물번호'].map(num_location_mapping)
train
le = LabelEncoder()
train['location'] = le.fit_transform(train['location'])
test['location'] = le.transform(test['location'])



## sin, cos
train['sin_time'] = np.sin(2*np.pi*train.hour/24)
train['cos_time'] = np.cos(2*np.pi*train.hour/24)
test['sin_time'] = np.sin(2*np.pi*test.hour/24)
test['cos_time'] = np.cos(2*np.pi*test.hour/24)
train['sin_day'] = np.sin(2*np.pi*train.day/7)
train['cos_day'] = np.cos(2*np.pi*train.day/7)
test['sin_day'] = np.sin(2*np.pi*test.day/7)
test['cos_day'] = np.cos(2*np.pi*test.day/7)



## 공휴일 변수 추가
train['holiday'] = train.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
train.loc[('2022-08-15'<=train.일시)&(train.일시<'2022-08-16'), 'holiday'] = 1

test['holiday'] = test.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
test.loc[('2022-08-15'<=test.일시)&(test.일시<'2022-08-16'), 'holiday'] = 1

## 빌딩별/날짜별/시간별 target mean, std
num_hour_mean = train.groupby(['건물번호', 'hour','day'])['전력소비량(kWh)'].mean()
num_hour_std = train.groupby(['건물번호', 'hour','day'])['전력소비량(kWh)'].std()
num_hour_mean = num_hour_mean.reset_index()
num_hour_std = num_hour_std.reset_index()
num_hour_mean.rename(columns={'전력소비량(kWh)': 'num_day_hour_mean'}, inplace=True)
num_hour_std.rename(columns={'전력소비량(kWh)': 'num_day_hour_std'}, inplace=True)
train = pd.merge(train, num_hour_mean, how='left', on=['건물번호', 'hour','day'])
train = pd.merge(train, num_hour_std, how='left', on=['건물번호', 'hour','day'])
test = pd.merge(test, num_hour_mean, how='left', on=['건물번호', 'hour','day'])
test = pd.merge(test, num_hour_std, how='left', on=['건물번호', 'hour','day'])

## 냉방도일
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)
cdhs = np.array([])
for num in range(1,101,1):
    temp = train[train['건물번호'] == num]
    cdh = CDH(temp['기온(C)'].values)
    cdhs = np.concatenate([cdhs, cdh])
train['CDH'] = cdhs
cdhs = np.array([])
for num in range(1,101,1):
    temp = test[test['건물번호'] == num]
    cdh = CDH(temp['기온(C)'].values)
    cdhs = np.concatenate([cdhs, cdh])
test['CDH'] = cdhs
'''
'''
features = ['건물기타', '공공', '대학교', '데이터센터', '백화점및아울렛',
            '병원', '상용', '아파트', '연구소', '지식산업센터', '할인마트', '호텔및리조트']
for feature in features:
    train[feature] = 0
    test[feature] = 0
num_ranges = [(1, 15, '건물기타'), (16, 23, '공공'), (24, 31, '대학교'),
                (32, 36, '데이터센터'), (37, 44, '백화점및아울렛'),
                (45, 52, '병원'), (53, 60, '상용'), (61, 68, '아파트'),
                (69, 76, '연구소'), (77, 84, '지식산업센터'),
                (85, 92, '할인마트'), (93, 100, '호텔및리조트')]
for start, end, feature in num_ranges:
    train.loc[(train['건물번호'] >= start) & (train['건물번호'] <= end), feature] = 1
    test.loc[(test['건물번호'] >= start) & (test['건물번호'] <= end), feature] = 1
'''
'''
# 건물 번호 범위를 정의
num_ranges = [(1, 15, '건물기타'), (16, 23, '공공'), (24, 31, '대학교'),
                (32, 36, '데이터센터'), (37, 44, '백화점및아울렛'),
                (45, 52, '병원'), (53, 60, '상용'), (61, 68, '아파트'),
                (69, 76, '연구소'), (77, 84, '지식산업센터'),
                (85, 92, '할인마트'), (93, 100, '호텔및리조트')]

# 라벨 인코딩 진행
for idx, (start, end, feature) in enumerate(num_ranges):
    train.loc[(train['건물번호'] >= start) & (train['건물번호'] <= end), '건물유형'] = idx
    test.loc[(test['건물번호'] >= start) & (test['건물번호'] <= end), '건물유형'] = idx



def calculate_commute_times(df):
    df['power_diff'] = df.groupby('건물번호')['전력소비량(kWh)'].diff()
    start_hour = df[(df['power_diff'] > 0) & (df['hour'].between(6,10))].groupby('건물번호')['power_diff'].idxmax().reset_index()
    start_hour.columns = ['건물번호', 'hour_idx']
    start_hour['start_hour'] = df.loc[start_hour['hour_idx'], 'hour'].values
    start_hour = start_hour.drop(columns='hour_idx')
    end_hour = df[(df['power_diff'] < 0) & (df['hour'].between(17,22))].groupby('건물번호')['power_diff'].idxmin().reset_index()
    end_hour.columns = ['건물번호', 'hour_idx']
    end_hour['end_hour'] = df.loc[end_hour['hour_idx'], 'hour'].values
    end_hour = end_hour.drop(columns='hour_idx')
    df = df.merge(start_hour, on='건물번호')
    df = df.merge(end_hour, on='건물번호')
    df = df.drop(columns='power_diff')
    return df
train = calculate_commute_times(train)
def calculate_commute_period(df):
    start_hour = df.groupby('건물번호')['start_hour'].first().reset_index()
    end_hour = df.groupby('건물번호')['end_hour'].first().reset_index()
    def is_commute_period(row):
        if row['hour'] >= start_hour.loc[row['건물번호'] - 1, 'start_hour'] and row['hour'] <= end_hour.loc[row['건물번호'] - 1, 'end_hour']:
            return 1
        else:
            return 0
    df['commute_period'] = df.apply(is_commute_period, axis=1)
    return df
train = calculate_commute_period(train)
num_commute_time_map = train.groupby('건물번호')[['start_hour', 'end_hour']].first().to_dict('index')
for num, times in num_commute_time_map.items():
    test.loc[test['건물번호'] == num, 'commute_period'] = ((test['hour'] >= times['start_hour']) & (test['hour'] <= times['end_hour'])).astype(int)
train.drop(['start_hour', 'end_hour'],axis=1,inplace=True)
test = test.copy()
test = test.sort_values(by=['건물번호', '일시'])
test = test.reset_index(drop = True)


## 체감온도
def body_temp(val):
    if val < 21:
        return 0
    elif 21 <= val < 25:
        return 1
    elif 25 <= val < 28:
        return 2
    elif 28 <= val < 31:
        return 3
    elif 31 <= val:
        return 4
    else:
        return None
train['body_temp'] = train['기온(C)'].apply(body_temp)
test['body_temp'] = test['기온(C)'].apply(body_temp)




## 빌딩별 쉬는날?
train['low_power_day'] = 0
# 건물기타
train.loc[(train['건물번호'].isin([2, 3]) & (train['day'] == 0)) | ((train['건물번호'] == 5) & (train['day'].between(0,3))), 'low_power_day'] = 1
# 공공
train.loc[(train['건물번호'].isin([17, 18, 19, 20, 21, 22, 23])) & (train['day'].isin([5, 6])), 'low_power_day'] = 1
# 대학교
train.loc[(train['건물번호'].isin([24, 25, 26, 27, 28, 29, 30, 31])) & (train['day'].isin([5, 6])), 'low_power_day'] = 1
# 병원
train.loc[(train['건물번호'].isin([45, 50]) & (train['day'] == 5)), 'low_power_day'] = 0.5
train.loc[(train['건물번호'].isin([45, 50]) & (train['day'] == 6)), 'low_power_day'] = 1
train.loc[(train['건물번호'].isin([46, 47, 48, 49, 51, 52])) & (train['day'].isin([5, 6])), 'low_power_day'] = 1
# 상용
train.loc[((train['건물번호'].isin([53, 55, 57, 58, 59, 60]) & (train['day'].isin([5, 6]))) | ((train['건물번호'] == 54) & (train['day'] == 0))), 'low_power_day'] = 1
# 연구소
train.loc[(train['건물번호'].isin([69, 70, 71, 72, 73, 74, 76]) & (train['day'].isin([5, 6]))), 'low_power_day'] = 1
# 지식산업센터
train.loc[(train['건물번호'].isin([77, 78, 79, 80, 82, 83, 84])) & (train['day'].isin([5, 6])), 'low_power_day'] = 1
# 일단 모든 값을 0으로 초기화
test['low_power_day'] = 0
# 건물기타
test.loc[(test['건물번호'].isin([2, 3]) & (test['day'] == 0)) | ((test['건물번호'] == 5) & (test['day'].between(0,3))), 'low_power_day'] = 1
# 공공
test.loc[(test['건물번호'].isin([17, 18, 19, 20, 21, 22, 23])) & (test['day'].isin([5, 6])), 'low_power_day'] = 1
# 대학교
test.loc[(test['건물번호'].isin([24, 25, 26, 27, 28, 29, 30, 31])) & (test['day'].isin([5, 6])), 'low_power_day'] = 1
# 병원
test.loc[(test['건물번호'].isin([45, 50]) & (test['day'] == 5)), 'low_power_day'] = 0.5
test.loc[(test['건물번호'].isin([45, 50]) & (test['day'] == 6)), 'low_power_day'] = 1
test.loc[(test['건물번호'].isin([46, 47, 48, 49, 51, 52])) & (test['day'].isin([5, 6])), 'low_power_day'] = 1
# 상용
test.loc[((test['건물번호'].isin([53, 55, 57, 58, 59, 60]) & (test['day'].isin([5, 6]))) | ((test['건물번호'] == 54) & (test['day'] == 0))), 'low_power_day'] = 1
# 연구소
test.loc[(test['건물번호'].isin([69, 70, 71, 72, 73, 74, 76]) & (test['day'].isin([5, 6]))), 'low_power_day'] = 1
# 지식산업센터
test.loc[(test['건물번호'].isin([77, 78, 79, 80, 82, 83, 84])) & (test['day'].isin([5, 6])), 'low_power_day'] = 1





## 전력 사용량의 차이 비율
power_diff_ratio_dict = {}
for num in range(1, 101):
    low_power_day_mean_power = train[(train['건물번호'] == num) & (train['low_power_day'] == 1)]['전력소비량(kWh)'].mean()
    not_low_power_day_mean_power = train[(train['건물번호'] == num) & (train['low_power_day'] == 0)]['전력소비량(kWh)'].mean()
    ambiguous_power_day_mean_power = train[(train['건물번호'] == num) & (train['low_power_day'] == 0.5)]['전력소비량(kWh)'].mean()

    if pd.isnull(low_power_day_mean_power) or pd.isnull(not_low_power_day_mean_power):
        power_diff_ratio = 0
    else:
        low_power_day_mean_power = (low_power_day_mean_power + ambiguous_power_day_mean_power / 2) if not pd.isnull(ambiguous_power_day_mean_power) else low_power_day_mean_power
        not_low_power_day_mean_power = (not_low_power_day_mean_power + ambiguous_power_day_mean_power / 2) if not pd.isnull(ambiguous_power_day_mean_power) else not_low_power_day_mean_power
        power_diff_ratio = abs(low_power_day_mean_power - not_low_power_day_mean_power) / not_low_power_day_mean_power
    power_diff_ratio_dict[num] = power_diff_ratio
train['power_diff_ratio'] = train['건물번호'].map(power_diff_ratio_dict)
test['power_diff_ratio'] = test['건물번호'].map(power_diff_ratio_dict)



# 32번, 33번만 해당하는 피처
train['power_increase_summer'] = 0
test['power_increase_summer'] = 0
# 7, 8월에 해당하는 32, 33번 num에 대해 'power_increase_summer' 값을 1로 설정
train.loc[(train['건물번호'].isin([32, 33])) & (train['month'].isin([7, 8])), 'power_increase_summer'] = 1
test.loc[(test['건물번호'].isin([32, 33])) & (test['month'].isin([7, 8])), 'power_increase_summer'] = 1



train.drop(['일조(hr)', '일사(MJ/m2)'],axis=1,inplace=True)
#test.drop(['일조(hr)', '일사(MJ/m2)'],axis=1,inplace=True)



train['증가'] = 0
train['감소'] = 0
for building_num, group in train.groupby('건물번호'):
    prev_power = group['전력소비량(kWh)'].shift(1)
    increase = (group['전력소비량(kWh)'] > prev_power).astype(int)
    decrease = (group['전력소비량(kWh)'] < prev_power).astype(int)
    train.loc[group.index, '증가'] = increase
    train.loc[group.index, '감소'] = decrease
summary_data = train.groupby(['건물번호', 'day', 'hour'])[['증가', '감소']].sum().reset_index()
summary_data.rename(columns={'증가': '증가_day_hour_sum', '감소': '감소_day_hour_sum'}, inplace=True)
train = pd.merge(train, summary_data, left_on=['건물번호', 'day', 'hour'], right_on=['건물번호', 'day', 'hour'], suffixes=('', '_day_hour_sum'))
test = pd.merge(test, summary_data, left_on=['건물번호', 'day', 'hour'], right_on=['건물번호', 'day', 'hour'], suffixes=('', '_day_hour_sum'))
train.drop(['증가','감소'], axis=1, inplace=True)
train['increase_all'] = 0
train['decrease_all'] = 0
train.loc[train['증가_day_hour_sum'] == 0, 'decrease_all'] = 1
train.loc[train['감소_day_hour_sum'] == 0, 'increase_all'] = 1
test['increase_all'] = 0
test['decrease_all'] = 0
test.loc[test['증가_day_hour_sum'] == 0, 'decrease_all'] = 1
test.loc[test['감소_day_hour_sum'] == 0, 'increase_all'] = 1




# 비가 오는 경우의 통계량 계산
rainy_stats = train[train['강수량(mm)'] > 0].groupby(['건물번호', 'day', 'hour'])['전력소비량(kWh)'].agg(['mean']).reset_index()
rainy_stats.columns = ['건물번호', 'day', 'hour', 'mean_rainy']

# 비가 오지 않는 경우의 통계량 계산
non_rainy_stats = train[train['강수량(mm)'] <= 0].groupby(['건물번호', 'day', 'hour'])['전력소비량(kWh)'].agg(['mean']).reset_index()
non_rainy_stats.columns = ['건물번호', 'day', 'hour', 'mean_non_rainy']

# 훈련 데이터에 비가 오는 경우와 오지 않는 경우의 통계 적용
train = pd.merge(train, rainy_stats, on=['건물번호', 'day', 'hour'], how='left')
train = pd.merge(train, non_rainy_stats, on=['건물번호', 'day', 'hour'], how='left')

# 비가 오는 인덱스에는 비가 올 때의 통계값을, 비가 오지 않는 인덱스에는 비가 오지 않을 때의 통계값을 넣어줌
train['mean'] = np.where(train['강수량(mm)'] > 0, train['mean_rainy'], train['mean_non_rainy'])


# 테스트 데이터에도 동일하게 적용
test = pd.merge(test, rainy_stats, on=['건물번호', 'day', 'hour'], how='left')
test = pd.merge(test, non_rainy_stats, on=['건물번호', 'day', 'hour'], how='left')

test['mean'] = np.where(test['강수량(mm)'] > 0, test['mean_rainy'], test['mean_non_rainy'])


# 필요 없는 칼럼 제거
train.drop(['mean_rainy', 'mean_non_rainy'], axis=1, inplace=True)
test.drop(['mean_rainy', 'mean_non_rainy'], axis=1, inplace=True)

train = train.sort_values(['건물번호','일시'])
train = train.reset_index(drop = True)
test = test.sort_values(['건물번호','일시'])
test = test.reset_index(drop = True)



train['연면_냉방면적'] = train['연면적'] / (train['냉방면적'] + 1)
test['연면_냉방면적'] = test['연면적'] / (test['냉방면적'] + 1)
train['태양광효율'] = train['연면적'] / (train['태양광용량'] + 1)
test['태양광효율'] = test['연면적'] / (test['태양광용량'] + 1)
train['태양광효율'] = train['연면적'] / (train['태양광용량'] + 1)
test['태양광효율'] = test['연면적'] / (test['태양광용량'] + 1)


def assign_rainy_category(group):
    rainy_count = (group['강수량(mm)'] > 0).sum()
    if rainy_count == 0:
        return 0
    elif rainy_count > 0 and rainy_count <= 3:
        return 1
    elif rainy_count > 3 and rainy_count <= 9:
        return 2
    else:
        return 3

rainy_categories = train.groupby(['건물번호', 'week', 'day']).apply(assign_rainy_category).reset_index(name='rainy_category')
train = pd.merge(train, rainy_categories, on=['건물번호', 'week', 'day'], how='left')

rainy_categories_test = test.groupby(['건물번호', 'week', 'day']).apply(assign_rainy_category).reset_index(name='rainy_category')
test = pd.merge(test, rainy_categories_test, on=['건물번호', 'week', 'day'], how='left')
train = train.sort_values(['건물번호','일시'])
test = test.sort_values(['건물번호','일시'])








# 폭염 경보, 폭염 주의보, 열대야 피처를 초기화합니다.
train['heatwave_alert'] = 0
train['heatwave_caution'] = 0
train['tropical_night'] = 0

# 빌딩 ID별로 루프를 돕니다.
for building_id in train['건물번호'].unique():
    # 해당 빌딩의 데이터만 선택
    building_data = train[train['건물번호'] == building_id]

    # 년월일별로 루프를 돕니다.
    for date in building_data['일시'].apply(lambda x: x.split(' ')[0]).unique():
        # 해당 빌딩, 날짜의 데이터만 선택
        daily_data = building_data[building_data['일시'].str.startswith(date)]

        # 최대 온도 계산
        max_temp = daily_data['기온(C)'].max()

        # 35도 이상이면 폭염 경보 1
        if max_temp >= 35:
            train.loc[(train['건물번호'] == building_id) & (train['일시'].str.startswith(date)), 'heatwave_alert'] = 1

        # 33도 이상이면 폭염 주의보 1
        if max_temp >= 33:
            train.loc[(train['건물번호'] == building_id) & (train['일시'].str.startswith(date)), 'heatwave_caution'] = 1

        # 모든 시간대의 온도가 25도 이상이면 열대야 1
        if daily_data['기온(C)'].min() >= 25:
            train.loc[(train['건물번호'] == building_id) & (train['일시'].str.startswith(date)), 'tropical_night'] = 1


test['heatwave_alert'] = 0
test['heatwave_caution'] = 0
test['tropical_night'] = 0

# 빌딩 ID별로 루프를 돕니다.
for building_id in test['건물번호'].unique():
    # 해당 빌딩의 데이터만 선택
    building_data = test[test['건물번호'] == building_id]

    # 년월일별로 루프를 돕니다.
    for date in building_data['일시'].apply(lambda x: x.split(' ')[0]).unique():
        # 해당 빌딩, 날짜의 데이터만 선택
        daily_data = building_data[building_data['일시'].str.startswith(date)]

        # 최대 온도 계산
        max_temp = daily_data['기온(C)'].max()

        # 35도 이상이면 폭염 경보 1
        if max_temp >= 35:
            test.loc[(test['건물번호'] == building_id) & (test['일시'].str.startswith(date)), 'heatwave_alert'] = 1

        # 33도 이상이면 폭염 주의보 1
        if max_temp >= 33:
            test.loc[(test['건물번호'] == building_id) & (test['일시'].str.startswith(date)), 'heatwave_caution'] = 1

        # 모든 시간대의 온도가 25도 이상이면 열대야 1
        if daily_data['기온(C)'].min() >= 25:
            test.loc[(test['건물번호'] == building_id) & (test['일시'].str.startswith(date)), 'tropical_night'] = 1
'''

In [3]:
X = train.drop(['전력소비량(kWh)'], axis=1)
y = train['전력소비량(kWh)']

In [4]:
def SMAPE(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output

In [5]:
import warnings
warnings.filterwarnings('ignore')

import optuna
from optuna import Trial

In [35]:
def objective(trial, model_r, model_name, X_train, y_train, X_valid, y_valid):
    if model_name == 'xgb':
        params = {
            "n_estimators": trial.suggest_int('n_estimators', 500, 5000),
            'max_depth': trial.suggest_int('max_depth', 8, 16),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
            'gamma': trial.suggest_int('gamma', 1, 3),
            'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            #'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
            #'alpha': trial.suggest_loguniform('alpha', 1e-3, 100.0),
            #'alpha': trial.suggest_loguniform('alpha', 1, 100.0),
            'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
            'alpha': trial.suggest_float('alpha', 1, 100.0, log=True),
            'subsample': trial.suggest_categorical('subsample', [0.6, 0.7, 0.8, 1.0]),
            'random_state': 724
        }
    elif model_name == 'lgbm':
        params = {
            'objective': 'regression', # 회귀
            'verbose': -1,
            'metric': 'rmse',
            'max_depth': trial.suggest_int('max_depth',3, 15),
            'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
            'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
            'random_state' : 724
          }
    elif model_name == 'cat':
        params = {
         'objective' :  'MAPE',
         #'n_estimators':trial.suggest_int("n_estimators", 30, 10000),
         'iterations':trial.suggest_int("iterations", 200, 250),
         'od_wait':trial.suggest_int('od_wait', 500, 2300),
         'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
         'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
         'random_strength': trial.suggest_uniform('random_strength',10,50),
         'subsample': trial.suggest_uniform('subsample',0.1,1),
         'random_state': trial.suggest_int('random_state', 0, 100),
         'depth': trial.suggest_int('depth',1, 15),
         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
         'random_state': 724,
         }

    elif model_name == 'rf':
        params = {
            'criterion': trial.suggest_categorical('criterion', ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']),
            'n_estimators':trial.suggest_int("n_estimators", 30, 400),
            'max_depth': trial.suggest_int('max_depth',1, 15),
            'min_samples_split': trial.suggest_int('min_samples_split',2,30),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf',1,30),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
            'min_weight_fraction_leaf' : trial.suggest_uniform('min_weight_fraction_leaf',0.0, 0.5),
            'bootstrap' : True,
            'oob_score' : True,
            'warm_start' : True,
            'verbose' : False,
            'random_state' : 724
        }
    elif model_name == 'et':
        params = {
              'criterion': trial.suggest_categorical('criterion', ['squared_error']),
              'n_estimators': trial.suggest_int('n_estimators', 1500, 3000),
              # 'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
              'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 15, 25),
              'bootstrap' : True,
              'oob_score' : True,
              'verbose' : 0,
              'warm_start' : True,
              'random_state' : 724
        }
    model = model_r.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    score = SMAPE(y_valid, y_pred)
    return score

In [36]:
# Adding functionality to save fold-wise predictions to CSV files
def get_stacking_base_datasets_with_txt_and_csv_backup(model, X_train_n, y_train_n, X_test_n, n_folds, model_name):

    # Create a stratified_target column in the DataFrame
    X_train_n['stratified_target'] = X_train_n['건물번호'].astype(str) + '_' + X_train_n['day'].astype(str) + '_' + X_train_n['hour'].astype(str)

    # Initialize StratifiedKFold
    kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=2)

    # Initialize empty arrays to hold prediction values
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))

    for folder_counter, (train_index, valid_index) in enumerate(kfold.split(X_train_n, X_train_n['stratified_target'])):
        print(f'Fold Set: {folder_counter} Start')

        X_tr = X_train_n.drop('stratified_target', axis=1).iloc[train_index]
        y_tr = y_train_n.iloc[train_index]
        X_val = X_train_n.drop('stratified_target', axis=1).iloc[valid_index]
        y_val = y_train_n.iloc[valid_index]

        # Hyperparameter tuning with Optuna for each fold
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective(trial, model, model_name, X_tr, y_tr, X_val, y_val), n_trials=25)

        best_params = study.best_params
        best_score = study.best_value
        tuned_model = model.set_params(**best_params)
        #tuned_model = eval(model_name)(**best_params)
        tuned_model.fit(X_tr, y_tr)

        # Save model name, hyperparameters and validation score to txt file
        with open("model_hyperparameters_and_score.txt", "a") as f:
            f.write(f"Model: {model_name}, Fold: {folder_counter}, Best Params: {best_params}, Best Score: {best_score}\n")

        # Prediction
        val_predictions = tuned_model.predict(X_val).reshape(-1, 1)
        test_predictions = tuned_model.predict(X_test_n)


        test_pred_df = pd.DataFrame(test_predictions, columns=['answer'])
        test_pred_df.to_csv(f"{model_name}_fold{folder_counter}_test_predictions.csv", index=False)

        train_fold_pred[valid_index, :] = val_predictions
        test_pred[:, folder_counter] = test_predictions

    # Drop the 'stratified_target' column before returning the results
    X_train_n.drop('stratified_target', axis=1, inplace=True)

    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)

    return train_fold_pred, test_pred_mean

In [37]:
XGB = XGBRegressor(tree_method='gpu_hist', gpu_id=0, random_state = 724)
lgbm = LGBMRegressor(random_state = 724)
Cat = CatBoostRegressor(random_state = 724)
et = ExtraTreesRegressor(random_state = 724)
rf = RandomForestRegressor(random_state = 724)

In [38]:
#xgb_train_pred, xgb_test_pred = get_stacking_base_datasets_with_txt_and_csv_backup(XGB, X, y, test, 7, 'xgb')
Cat_train_pred, Cat_test_pred = get_stacking_base_datasets_with_txt_and_csv_backup(Cat, X, y, test, 7, 'cat')
lgbm_train_pred, lgbm_test_pred = get_stacking_base_datasets_with_txt_and_csv_backup(lgbm, X, y, test, 7, 'lgbm')
et_train_pred, et_test_pred = get_stacking_base_datasets_with_txt_and_csv_backup(et, X, y, test, 7, 'et')
rf_train_pred, rf_test_pred = get_stacking_base_datasets_with_txt_and_csv_backup(rf, X, y, test, 7, 'rf')

[I 2023-08-26 22:21:03,397] A new study created in memory with name: no-name-ddd3ffe5-cf89-4752-aac6-4739212c91fb


Fold Set: 0 Start
0:	learn: 0.5192132	total: 7.57ms	remaining: 1.55s
1:	learn: 0.5174648	total: 14.4ms	remaining: 1.47s
2:	learn: 0.5147103	total: 21.2ms	remaining: 1.44s
3:	learn: 0.4963204	total: 28.5ms	remaining: 1.44s
4:	learn: 0.4906098	total: 35.7ms	remaining: 1.44s
5:	learn: 0.4893931	total: 42.4ms	remaining: 1.41s
6:	learn: 0.4881287	total: 49ms	remaining: 1.39s
7:	learn: 0.4870623	total: 55.6ms	remaining: 1.37s
8:	learn: 0.4566576	total: 63ms	remaining: 1.38s
9:	learn: 0.4546039	total: 69.8ms	remaining: 1.37s
10:	learn: 0.4528537	total: 77.5ms	remaining: 1.37s
11:	learn: 0.4514016	total: 83.4ms	remaining: 1.35s
12:	learn: 0.4506186	total: 89.3ms	remaining: 1.33s
13:	learn: 0.4474383	total: 96.2ms	remaining: 1.32s
14:	learn: 0.4466360	total: 103ms	remaining: 1.31s
15:	learn: 0.4428350	total: 110ms	remaining: 1.31s
16:	learn: 0.4419535	total: 117ms	remaining: 1.3s
17:	learn: 0.4410032	total: 124ms	remaining: 1.29s
18:	learn: 0.4401562	total: 130ms	remaining: 1.28s
19:	learn: 0.4

[I 2023-08-26 22:21:04,928] Trial 0 finished with value: 51.35071181523614 and parameters: {'iterations': 206, 'od_wait': 527, 'learning_rate': 0.5438513125520766, 'reg_lambda': 9.601929159868257, 'random_strength': 41.094016990399155, 'subsample': 0.4271001330366796, 'random_state': 10, 'depth': 1, 'min_data_in_leaf': 24}. Best is trial 0 with value: 51.35071181523614.
[W 2023-08-26 22:21:04,931] Trial 1 failed with parameters: {'iterations': 239, 'od_wait': 1422, 'learning_rate': 0.6571466639432655, 'reg_lambda': 53.244217492363525, 'random_strength': 28.576250913872556, 'subsample': 0.701411426119777, 'random_state': 90, 'depth': 11, 'min_data_in_leaf': 11} because of the following error: CatBoostError("You can't change params of fitted model.").
Traceback (most recent call last):
  File "c:\Users\ineeji\anaconda3\envs\v1\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ineeji\AppData\Local\Temp\ipykernel_36116\

194:	learn: 0.3705074	total: 1.31s	remaining: 74.1ms
195:	learn: 0.3701690	total: 1.32s	remaining: 67.4ms
196:	learn: 0.3698666	total: 1.33s	remaining: 60.6ms
197:	learn: 0.3698242	total: 1.33s	remaining: 53.9ms
198:	learn: 0.3697440	total: 1.34s	remaining: 47.2ms
199:	learn: 0.3694254	total: 1.35s	remaining: 40.4ms
200:	learn: 0.3690470	total: 1.35s	remaining: 33.7ms
201:	learn: 0.3689875	total: 1.36s	remaining: 26.9ms
202:	learn: 0.3689286	total: 1.37s	remaining: 20.2ms
203:	learn: 0.3688745	total: 1.37s	remaining: 13.5ms
204:	learn: 0.3688165	total: 1.38s	remaining: 6.74ms
205:	learn: 0.3687620	total: 1.39s	remaining: 0us


CatBoostError: You can't change params of fitted model.

In [ ]:
# 개별 모델로부터 나온 y_train 예측값들 옆으로 붙이기
Stack_final_X_train = np.concatenate((xgb_train_pred,Cat_train_pred,lgbm_train_pred,et_train_pred, rf_train_pred), axis=1)
# 개별 모델로부터 나온 y_test 예측값들 옆으로 붙이기
Stack_final_X_test = np.concatenate((xgb_test_pred,Cat_test_pred,lgbm_test_pred,et_test_pred, rf_test_pred), axis=1)


lr_final = Ridge(alpha = 0.9)
lr_final.fit(Stack_final_X_train, y)
stack_final = lr_final.predict(Stack_final_X_test)

submit['target'] = stack_final
submit.to_csv('stacking_meta_ridge9555.csv', index = False)